In [14]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
#! pip install plotly
import plotly.express as px 

In [15]:
from pathlib import Path

data_folder = Path("/Users/shengshenghuo/Downloads/Rice_MSC_Dataset") 
# make sure to change this path to the actual path where YOU have saved the data file. 

file_to_open = data_folder / "Rice_MSC_Dataset.xlsx"

f = open(file_to_open)



In [16]:
! pip install openpyxl

In [17]:
df1 = pd.read_excel("Rice_MSC_Dataset.xlsx")

In [18]:
df1.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,ALLdaub4L,ALLdaub4a,ALLdaub4b,ALLdaub4Y,ALLdaub4Cb,ALLdaub4Cr,ALLdaub4XX,ALLdaub4YY,ALLdaub4ZZ,CLASS
0,7805,437.915,209.8215,48.0221,0.9735,99.6877,0.9775,7985,0.3547,4.3693,...,113.9924,65.0610,59.5989,104.8552,67.8779,63.0828,0.3673,0.3793,0.4733,Basmati
1,7503,340.757,138.3361,69.8417,0.8632,97.7400,0.9660,7767,0.6637,1.9807,...,105.7055,64.3685,62.2084,96.8375,65.5371,63.5832,0.3014,0.3144,0.3641,Arborio
2,5124,314.617,141.9803,46.5784,0.9447,80.7718,0.9721,5271,0.4760,3.0482,...,109.7155,62.6423,58.7439,100.2352,68.9753,59.8342,0.3233,0.3445,0.4448,Jasmine
3,7990,437.085,201.4386,51.2245,0.9671,100.8622,0.9659,8272,0.6274,3.9325,...,116.5405,64.9069,60.2562,107.2560,67.3298,63.2237,0.3880,0.4020,0.4904,Basmati
4,7433,342.893,140.3350,68.3927,0.8732,97.2830,0.9831,7561,0.6006,2.0519,...,107.7502,64.7071,61.3549,98.8704,66.2048,63.5378,0.3184,0.3303,0.3928,Arborio


In [19]:
df1.isnull().sum()

AREA            0
PERIMETER       0
MAJOR_AXIS      0
MINOR_AXIS      0
ECCENTRICITY    0
               ..
ALLdaub4Cr      0
ALLdaub4XX      0
ALLdaub4YY      0
ALLdaub4ZZ      0
CLASS           0
Length: 107, dtype: int64

In [20]:
y = df1['CLASS']
types = y.unique()
y= y.values.reshape(-1,1)
# print(y)
# type(y)

x = df1.drop(['CLASS'],axis=1)
x = df1.iloc[:,:16]
x = x.values
# print(x)
# type(x)

df1.dropna(inplace =True)
# take the random sample 
df1 = df1.sample(n=7500, random_state=1)
y = df1.CLASS 
x = df1.iloc[:,:16]

# normaliize the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
xnorm = sc.fit_transform(x)
xnorm
#print(xnorm)



# perform PCA

xbar = np.mean(xnorm)
xtilde = xnorm - xbar
sample_scatter = np.dot(xtilde.T, xtilde)

# method 1 
eigenvalues, eigenvectors = np.linalg.eig(sample_scatter)
# sorting the eigenvalues and eigenvectors 
index = eigenvalues.argsort()[::-1]
eigenvalues = eigenvalues[index]
eigenvectors = eigenvectors[:,index]
projected = xtilde @ eigenvectors # PCA projection


#method 2
covariance_scatteer = xtilde.T @ xtilde
U,S,V = np.linalg.svd(covariance_scatteer)
projected = xtilde @ U # PCA projection





In [21]:

projected.shape
pca1 = projected[:,0]
pca2 = projected[:,1]
pca3 = projected[:,2]
fig = px.scatter_3d(data_frame = xtilde, x = pca1, y = pca2, z = pca3, color = y, template = 'plotly_dark', color_discrete_sequence=px.colors.qualitative.Pastel2)




fig.update_layout(scene=dict(xaxis_title='PCA1',
                  yaxis_title='PCA2', zaxis_title='PCA3'))
#resize the plot
fig.update_layout(width=1000, height=1000)
#make points smaller
fig.update_traces(marker=dict(
    size=3), selector=dict(mode='markers'))



fig.show()

#### Perform KMeans after PCA

In [22]:
from sklearn.cluster import KMeans
km = KMeans(n_clusters = 5, n_init= 10)
kmeans_cluster = km.fit_predict(xnorm)
fig = px.scatter_3d(data_frame= xtilde, x = pca1, y = pca2, z = pca3, color = kmeans_cluster,hover_name=y, template = 'plotly_dark')




fig.update_layout(scene=dict(xaxis_title='PCA1',
                  yaxis_title='PCA2', zaxis_title='PCA3'))
#resize the plot
fig.update_layout(width=1000, height=1000)
#make points smaller
fig.update_traces(marker=dict(
    size=3), selector=dict(mode='markers'))

fig.show()

##### Perform LDA after PCA

In [23]:
xnorm.shape[0] # number of rows
local_central_matrix = np.zeros((xnorm.shape[0],5))
Ytilde = projected
size = np.zeros((5,1))

In [24]:
def lda(X, labels):
    n,d = X.shape
    levels = np.unique(labels)
    c = len(levels)
    local_centroids = np.zeros((c+1,d))
    Ytilde = np.zeros((n,d))
    group_counts = np.zeros((c,1))
    for i in range(c):
        inds_i = np.flatnonzero(labels == levels[i])
        group_counts[i] = np.sum(inds_i)
        local_centroids[i,:] = np.mean(X[inds_i,:],axis=0)
        Ytilde[inds_i,:] = X[inds_i,:]-local_centroids[i,:]
    Sw = Ytilde.T @ Ytilde
    #Sb = groupcounts * sum((local_centroids - global_centroids)@ (local_centroids - global_centroids).T)
    Sb = np.zeros((d,d))
    for i in range(c):
        Sb += group_counts[i] * np.outer(local_centroids[i,:]-np.mean(X,axis=0),local_centroids[i,:]-np.mean(X,axis=0))
    _, eigenvectors = np.linalg.eigh(np.linalg.pinv(Sw).dot(Sb))
    pc = X.dot(eigenvectors[:,::-1][:,:5])
    return pc, eigenvectors

# perform LDA for PCA and CLASS
Zlda, V = lda(projected, y)    

In [25]:
fig =px.scatter_3d(data_frame = xtilde, x = Zlda[:,0],y = Zlda[:,1],z = Zlda[:,2], color = y,template = 'plotly_dark')

fig.update_layout(scene=dict(xaxis_title='PCA1',
                  yaxis_title='PCA2', zaxis_title='PCA3'))
#resize the plot
fig.update_layout(width=1000, height=1000)
#make points smaller
fig.update_traces(marker=dict(
    size=3), selector=dict(mode='markers'))





fig.show()

#### perform spectral clustering


In [26]:
X=xnorm
n = X.shape[0]# number of rows

# # Method 2
from sklearn.metrics.pairwise import pairwise_distances
# construct the weigthed matrix w
dist = pairwise_distances(X, metric = 'euclidean')



subset = np.random.choice(n,300,replace = False)

# find the k - nearest neighbor of dataset 
from sklearn.neighbors import NearestNeighbors
ngh = 8

nbrs = NearestNeighbors(n_neighbors=ngh).fit(X)

knnDistances, knnIndices = nbrs.kneighbors(X[subset,:],return_distance=True)


sigma = np.mean(knnDistances[:,-1])# last column of distance matric 

W = np.exp(-dist/(2*sigma**2))

# set diagonal of W matrix to be 0 
W = W - np.diag(np.diag(W))

P = W/np.sum(W, axis = 1)


from scipy.sparse.linalg import eigsh

eigvalues, eigvectors = eigsh(P, k = 6, which = 'LM')

# sort the eigenvalues and eigenvectors based the index of eigenvalues
index = np.argsort(eigvalues)[::-1]
# sorted 
eigvalues = eigvalues[index]
eigvectors = eigvectors[:,index]

# find the 2nd to kth largest eigenvectors V = [v2,.....vk] of P 
V = eigvectors

# apply kmeans 
kmeans = KMeans(n_clusters=5, n_init = 10)

spectral_clustering = kmeans.fit_predict(V[:,1:ngh]) 


# plot it 
pca1 = projected[:,0]
pca2 = projected[:,1]
pca3 = projected[:,2]
fig = px.scatter_3d(data_frame = xtilde, x = pca1, y = pca2, z = pca3, color = spectral_clustering,hover_name = y, template = 'plotly_dark')


fig.update_layout(scene=dict(xaxis_title='PCA1',
                  yaxis_title='PCA2', zaxis_title='PCA3'))
#resize the plot
fig.update_layout(width=1000, height=1000)
#make points smaller
fig.update_traces(marker=dict(
    size=3), selector=dict(mode='markers'))
fig.show()

### error function 

In [27]:
from scipy.optimize import linear_sum_assignment

import sys 
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
Y = le.fit_transform(y)
def Hungarian(A):
    _, col_ind = linear_sum_assignment(A)
    return col_ind
def clustering_error(L1,L2):
    '''
    Input: L1 = True Labels 
           L2 = Cluster Labels
    Output: Error Rate
    '''
    L1 = L1.flatten(order = 'F').astype(float)
    L2 = L2.flatten(order = 'F').astype(float)
    n = L1.size
    if L1.size != L2.size:
        sys.exit('Size of L1 and L2 must be equal.')
    #unique elements in L1 
    Label1 = np.unique(L1)
    #how many unique elements in L1
    nClass1 = Label1.size
    #unique elements in L2
    Label2 = np.unique(L2)
    #how many unique elements in L2
    nClass2 = Label2.size
    #maximum number of clusters
    nClass = max(nClass1,nClass2)
    
    G = np.zeros((nClass,nClass)).astype(float)
    for i in range(0,nClass2):
        for j in range(0,nClass1):
            #number of elements in L1 that have label Label1[j] and L2 that have label Label2[i]
            G[i,j]= np.sum(np.logical_and(L2 == Label2[i], L1 == Label1[j]))
    #find the optimal permutation
    c = Hungarian(-G)
    #find the permuted matrix
    newL2 = np.zeros(L2.shape)
    #permute the matrix
    for i in range(0,nClass2):
        #find the index of the permuted matrix
        newL2[L2 == Label2[i]] = Label1[c[i]]
        
    #find the error rate    
    return 1- (np.sum(newL2 == L1))/n

In [ ]:
lustering_error = cle